In [42]:
# %load backtester.py
#!/usr/bin/env python3

import logging
import pandas as pd

from argparser import convert_args_to_trading_variables, get_parsed_args
from binance_api_downloader import get_data
from functools import reduce


def main():
    logging_setup()
    args = {'end_date': '2022-04-04',
 'interval': '1d',
 'pairs': ['BTCUSDT', 'ETHUSDT', 'ADAUSDT', 'SOLUSDT'],
 'start_date': '2016-01-01'}

    trading_vars = convert_args_to_trading_variables(args)

    pair_list = []
    for pair in trading_vars.pairs:
        args = {
            "ticker": pair,
            "start_date": trading_vars.start_date,
            "end_date": trading_vars.end_date,
            "interval": args["interval"],
        }

        data = get_data(args)

        if (args['interval'] == '1d'):
            # normalize the datetime to midnight if interval is 1 day
            data.index = data.index.map((lambda x: pd.Timestamp(x).replace(hour=0, minute=0, second=0)))

        multinindex = pd.MultiIndex.from_product([[pair], data.index.values], names=["pair", "open_time"])
        df = pd.DataFrame(data.values, columns=['open', 'high', 'low', 'close', 'volume'], index=multinindex)
        pair_list.append(df)

    data = pd.concat(pair_list)
    data = remove_distinct_dates(data)
    data = data.sort_index()

    return data


def logging_setup():
    logging.basicConfig(
        level=logging.INFO,
        format="[%(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        handlers=[logging.FileHandler("lastrun.log", mode="w"), logging.StreamHandler()],
    )
    
    
def get_symbols_from_index(data):
    return data.index.get_level_values(level="pair").unique()


def get_dates_from_index(data):
    return data.index.get_level_values(level="open_time").unique()

def remove_distinct_dates(data: pd.DataFrame):
    coins = get_symbols_from_index(data)
    for coin in coins:
        logging.info(f"{coin}: {data.loc[coin].reset_index().iloc[0]['open_time']}")

    dates = []
    for coin in set(data.index.get_level_values(level="pair")):
        dates.append(data.loc[coin].index.values)
    dates = reduce(np.intersect1d, dates)

    df = data.reset_index()
    filtr = (df['open_time'] < dates[0]) | (df['open_time'] > dates[-1])
    df = df.drop(df.index[filtr])
    df = df.set_index(['open_time', 'pair'])
    return df


data = main()
data
price = data['close']


import vectorbt as vbt
import plotly.express as px
import plotly.graph_objects as go
import functools
import numpy as np

df = data

df
df.reset_index()
df = df.reset_index().set_index(['open_time', 'pair']).sort_index()
df = df.reset_index().set_index(['pair', 'open_time']).sort_index()
df.loc['BTCUSDT'].close
dates = get_dates_from_index(df)
a = map(lambda x: list(dates).index(x), [pd.Timestamp('2020-10-11'), pd.Timestamp('2021-01-01')])
list(a)

[INFO] opening existing csv file: ../data/binance/BTCUSDT:2000-01-01:2022-04-04:1d.csv
[INFO] opening existing csv file: ../data/binance/ETHUSDT:2000-01-01:2022-04-04:1d.csv
[INFO] opening existing csv file: ../data/binance/ADAUSDT:2000-01-01:2022-04-04:1d.csv
[INFO] opening existing csv file: ../data/binance/SOLUSDT:2000-01-01:2022-04-04:1d.csv
[INFO] BTCUSDT: 2017-08-17 00:00:00
[INFO] ETHUSDT: 2017-08-17 00:00:00
[INFO] ADAUSDT: 2018-04-17 00:00:00
[INFO] SOLUSDT: 2020-08-11 00:00:00


[61, 143]